In [1]:
#import nessesary library for data pre-processing
from keras.preprocessing.image import ImageDataGenerator

#Use ImageDataGenerator for data augmentation to get more data for training and rescaling validation and testing data
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=50, zoom_range=0.4, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 10
train_generator = train_datagen.flow_from_directory('train', target_size=(244,244), batch_size=batch_size, class_mode='binary', seed=1)
valid_generator = valid_datagen.flow_from_directory('valid', target_size=(244,244), batch_size=batch_size, class_mode='binary', seed=1)

Using TensorFlow backend.


Found 161 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(244, 244, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 242, 242, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 242, 242, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 121, 121, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 119, 119, 32)      4640      
_________________________________________________________________
activation_2 (Activation)    (None, 119, 119, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 59, 59, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 57, 57, 64)        18496     
__________

In [3]:
from keras.callbacks import ModelCheckpoint

#use ModelCheckpoint to save weights with the best accuracy
checkpointer = ModelCheckpoint(filepath='weights.best.h5py', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

#train the network on train_generator dataset
model.fit_generator(train_generator, steps_per_epoch=len(train_generator.filenames), epochs=7, validation_data=valid_generator,\
                    validation_steps=len(valid_generator.filenames),callbacks=[checkpointer], verbose=1)


Epoch 1/7
161/161 [==============================] - 330s 2s/step - loss: 0.5922 - acc: 0.7726 - val_loss: 0.3425 - val_acc: 0.8333

Epoch 00001: val_acc improved from -inf to 0.83333, saving model to weights.best.h5py
Epoch 2/7
161/161 [==============================] - 320s 2s/step - loss: 0.3288 - acc: 0.8497 - val_loss: 0.4282 - val_acc: 0.7778

Epoch 00002: val_acc did not improve from 0.83333
Epoch 3/7
161/161 [==============================] - 319s 2s/step - loss: 0.2682 - acc: 0.8963 - val_loss: 0.6247 - val_acc: 0.8333

Epoch 00003: val_acc did not improve from 0.83333
Epoch 4/7
161/161 [==============================] - 321s 2s/step - loss: 0.2288 - acc: 0.9236 - val_loss: 0.3181 - val_acc: 0.9259

Epoch 00004: val_acc improved from 0.83333 to 0.92593, saving model to weights.best.h5py
Epoch 5/7
161/161 [==============================] - 317s 2s/step - loss: 0.1791 - acc: 0.9397 - val_loss: 1.2007 - val_acc: 0.7593

Epoch 00005: val_acc did not improve from 0.92593
Epoch 6/7


In [10]:
import numpy as np

#Use this function to get labels for the augmentid data
def get_labels(data_generator):
    labels = []
    for i in range(len(data_generator)):
        label = data_generator[i][1]
        for j in range(len(label)):
            labels.append(label[j])
    return np.array(labels).reshape((len(data_generator.filenames),1))

In [15]:
#Use VGG16 architecture, pre-trained on the ImageNet dataset
from keras.applications import VGG16

#Instantiate only the convolutional part of the model,
model_VGG16 = VGG16(weights="imagenet", include_top=False)

train_generator = train_datagen.flow_from_directory('train', target_size=(244,244), batch_size=batch_size, class_mode='binary', seed=1)
valid_generator = valid_datagen.flow_from_directory('valid', target_size=(244,244), batch_size=batch_size, class_mode='binary', seed=1)

train_labels = get_labels(train_generator)
valid_labels = get_labels(valid_generator)

train_data = model_VGG16.predict_generator(train_generator, len(train_generator.filenames)/batch_size)
valid_data = model_VGG16.predict_generator(valid_generator, len(valid_generator.filenames)/batch_size)


Found 161 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [20]:

#And here run the rest of the VGG16 model on our dataset
transfer_model = Sequential()
transfer_model.add(Flatten(input_shape=train_data.shape[1:]))
transfer_model.add(Dense(256))
transfer_model.add(Activation('relu'))
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(1))
transfer_model.add(Activation('sigmoid'))

transfer_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               6422784   
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 257       
_________________________________________________________________
activation_11 (Activation)   (None, 1)                 0         
Total params: 6,423,041
Trainable params: 6,423,041
Non-trainable params: 0
_________________________________________________________________


In [25]:
#use ModelCheckpoint to save weights with the best accuracy
transfer_checkpointer = ModelCheckpoint(filepath='transfer_weights.best.h5py', monitor='val_acc', verbose=1, save_best_only=True, mode='max')

transfer_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

#train the network on train_data (predictions from convolutional part of the model)
transfer_model.fit(train_data, train_labels, epochs=40, validation_data=(valid_data, valid_labels), callbacks=[transfer_checkpointer], verbose=1)

Train on 161 samples, validate on 54 samples
Epoch 1/40
161/161 [==============================] - 2s 13ms/step - loss: 4.2460e-07 - acc: 1.0000 - val_loss: 0.4784 - val_acc: 0.9444

Epoch 00001: val_loss improved from inf to 0.47840, saving model to transfer_weights.best.h5py
Epoch 2/40
161/161 [==============================] - 1s 7ms/step - loss: 0.0185 - acc: 0.9938 - val_loss: 1.7158 - val_acc: 0.8333

Epoch 00002: val_loss did not improve from 0.47840
Epoch 3/40
161/161 [==============================] - 1s 6ms/step - loss: 0.0131 - acc: 0.9938 - val_loss: 0.4948 - val_acc: 0.9444

Epoch 00003: val_loss did not improve from 0.47840
Epoch 4/40
161/161 [==============================] - 1s 7ms/step - loss: 1.1338e-07 - acc: 1.0000 - val_loss: 0.4948 - val_acc: 0.9444

Epoch 00004: val_loss did not improve from 0.47840
Epoch 5/40
161/161 [==============================] - 1s 7ms/step - loss: 7.4180e-07 - acc: 1.0000 - val_loss: 0.4922 - val_acc: 0.9444

Epoch 00005: val_loss did not

In [3]:
#Change path to your training dataset and run this cell
path = 'test'
batch_size = 10

#import nessesary library for data pre-processing
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16

model_VGG16 = VGG16(weights="imagenet", include_top=False)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(path, target_size=(244,244), batch_size=batch_size, class_mode='binary',seed=1)

labels = []
for i in range(len(test_generator)):
    label = test_generator[i][1]
    for j in range(len(label)):
        labels.append(label[j])

test_labels = np.array(labels)
test_data = model_VGG16.predict_generator(test_generator, len(test_generator.filenames)/batch_size)

model = Sequential()
model.add(Conv2D(16, (3, 3), input_shape=(244, 244, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


transfer_model = Sequential()
transfer_model.add(Flatten(input_shape=test_data.shape[1:]))
transfer_model.add(Dense(256))
transfer_model.add(Activation('relu'))
transfer_model.add(Dropout(0.5))
transfer_model.add(Dense(1))
transfer_model.add(Activation('sigmoid'))
transfer_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
 
#load the weights with the best accuracy value
model.load_weights('weights.best.h5py')
transfer_model.load_weights('transfer_weights.best.h5py') 

#Evaluate the models on the test set
scores = model.evaluate_generator(test_generator,54)
transfer_scores = transfer_model.evaluate(test_data, test_labels)

#print the results
print ("Accuracy: ", scores[1])
print ("Transfer Learning Accuracy: ", transfer_scores[1])

Found 54 images belonging to 2 classes.
54/54 [==============================] - 0s 6ms/step
Accuracy:  0.8333333296540343
Transfer Learning Accuracy:  0.7962963051266141
